In [101]:
import cv2
import numpy as np
from detection2 import Yaw


name = "IMG_3603"
# name = "IMG_3468"

cap = cv2.VideoCapture('sh/mov/'+ name +'.MOV')
frames = []
angles = []
yaw_obj = Yaw()


c= 1
while True:


    ret, frame = cap.read()
    if not ret:
        break
    yaww = yaw_obj.predict_yaw(frame)
    angles.append(yaww)
    frames.append(frame)

    if (c% 100 ==0):
        print(c , yaww)
    c+=1

    
cap.release()
video_array = np.array(frames)

Using cache found in C:\Users\admin/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2023-5-2 Python-3.10.11 torch-2.0.0 CUDA:0 (NVIDIA GeForce RTX 3090, 24575MiB)

Fusing layers... 


requirements: C:\Users\admin\.cache\torch\hub\requirements.txt not found, check failed.


YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


Model: "model_16"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_17 (InputLayer)          [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 block1_conv1 (Conv2D)          (None, 224, 224, 64  1792        ['input_17[0][0]']               
                                )                                                                 
                                                                                                  
 block1_conv2 (Conv2D)          (None, 224, 224, 64  36928       ['block1_conv1[0][0]']           
                                )                                                          

In [102]:
import os
import random
r = random.randint(0,12000)
# Path
# Create the directory
# 'GeeksForGeeks' in
# '/home / User / Documents'
os.mkdir("sh/mov/"+name+"__"+ str(r))

In [103]:
len(angles)

1686

In [104]:
def find_nearest(lst, num):
    nearest_num = lst[0]
    nearest_idx = 0
    min_diff = abs(num - lst[0])
    
    for i in range(1, len(lst)):
        diff = abs(num - lst[i])
        if diff < min_diff:
            nearest_num = lst[i]
            nearest_idx = i
            min_diff = diff
            
    return (nearest_num, nearest_idx)

In [105]:
def rotate_180(angle):
    return (angle + 180) % 360

In [106]:
def select_frame_from_period(frames ,angles , angle_range):
    print(angle_range ," -------> ", angles)
    a , b = angle_range
    angle_avg = (a+b) / 2
    angle , idx = find_nearest(angles , angle_avg)
    if 280 > abs(angle - angle_avg) > 100:
        angle_avg = rotate_180(angle_avg)
        angle , idx = find_nearest(angles , angle_avg)
        angle = rotate_180(angle)
    return frames[idx] , angle





In [107]:
def frame_selection(angles , frames , num_frames = 30):
    selected_frames = []
    selected_angles = []
    angle_period = 360 // num_frames
    frame_period = frames.shape[0] // num_frames
    for i in range(num_frames):
        # print(i)
        half_period = angle_period // 2
        a = i * angle_period - half_period
        b = (i+1) * angle_period - half_period - 1
        c = i * frame_period
        d = (i+1) * frame_period -1
        

        frame , angle = select_frame_from_period( frames[c:d,:,:,:],angles[c:d] , angle_range = (a,b))

        selected_frames.append(frame)
        selected_angles.append(angle)

    return selected_frames , selected_angles




        

In [108]:
sel_frames , sel_angles = frame_selection(angles , video_array , 25)

(-7, 6)  ------->  [1.060693161510585, 180.53195491037079, 1.481045138007346, 1.397146052451376, 1.488426013818298, 1.441947733836173, 1.6910439681607272, 1.7046448006520905, 1.4418119837778818, 1.8425688875912556, 179.67921599967394, 1.6345528407185839, 1.4733618127739874, 1.5981101399279396, 2.3086485700450226, 1.7743951442832895, 1.9803771814382571, 2.048194794286701, 1.8733256673082224, 2.7198463822334134, 2.708451701632839, 2.113931333128278, 1.6248263276974626, 1.9171086896470797, 1.901219956409136, 2.04714358510887, 2.1858483416049808, 2.6202186460553776, 2.555244313909043, 2.346488471907321, 3.0757130256145615, 180.05598511046253, 180.05937922877587, 180.4558866928591, 180.23576623910813, 180.24947830233714, 180.34438194715872, 180.3907029993316, 180.14130643667937, 180.31219274819463, 180.23242230002478, 180.21032154753613, 180.31281896470765, 180.47932193405467, 180.42200504711874, 180.6259207912284, 180.48741869932402, 180.42477818495462, 180.1083845796029, 180.2130947120524

In [109]:
sel_angles

[1.060693161510585,
 13.516963977422506,
 23.514119926028627,
 41.25578173343314,
 56.27110635343416,
 66.48248998129736,
 94.11366306372511,
 107.21794062488306,
 126.29903712051141,
 136.67825264355767,
 154.98100444662057,
 161.6512927354821,
 180.53452370274744,
 181.50288286975632,
 195.35942215371216,
 199.45883827256046,
 241.31142031196714,
 242.15841465799198,
 252.03282800789984,
 300.3773701873822,
 301.5043588203571,
 304.4541179408065,
 316.370083461023,
 336.10271538753426,
 344.0181100412539]

In [110]:
sel_frames = np.array(sel_frames)
size = sel_frames.shape[2] , sel_frames.shape[1]

fps = 2


out = cv2.VideoWriter('sh/mov/'+ name+"__"+ str(r) +'/reco_.mp4', cv2.VideoWriter_fourcc(*'mp4v'), fps, size)

for f in sel_frames:
    out.write(f)
out.release() 


In [111]:

# List of images
images = sel_frames # Replace with your list of NumPy arrays

# Write images to disk
for i, image in enumerate(images):
    filename = f"sh/mov/"+ name+"__"+ str(r) +f"/image_{i}.png"
    cv2.imwrite(filename, image)

## real time detection

In [112]:
def find_main_angle(angle , expectation_idx = 0): # expectation_idx is previous angle_idx
    important_angles = [0 , 30 , 90  , 150 , 180 , 200 , 270 , 340 , 360]
    angle_names = ["front" , "front-left" , "left" , "back-left" , "back" , "back-right" , "right" , "front-right" , "front"]
    ang , idx = find_nearest(important_angles , angle)
    if 280 > abs(ang - important_angles[expectation_idx]) > 100: # dont mistake on 180 degree
        angle2 = rotate_180(angle)
        ang , idx = find_nearest(important_angles , angle2)
    if idx != expectation_idx + 1 : # dont turn back
        idx = expectation_idx
    return angle_names[idx] , idx
    
    


In [113]:
font = cv2.FONT_HERSHEY_SIMPLEX
x, y = 50, 100
color = (150, 35, 200)
font_scale = 1
thickness = 4



In [114]:
import cv2
import os
import copy
# Set the directory containing the images
li = copy.deepcopy(video_array)

# Create a window to display the video
cv2.namedWindow('Video', cv2.WINDOW_NORMAL)

# Loop through each image and display it in the window
c= 0 
yaww = 0 
expectation_idx = 0
yawObj = Yaw()

detected_video = []

for img in li:
    if c % 20 == 0 :
        yaww = yawObj.predict_yaw(img)
        ang , idx = find_main_angle(yaww , expectation_idx = expectation_idx)
        expectation_idx = idx
    cv2.putText(img, "angle: "+str(ang), (x,y), font, font_scale, color, thickness)
    cv2.imshow('Video', img)
    detected_video.append(img)
    c+=1
    # Wait for 25 milliseconds between each frame (40 frames per second)
    if cv2.waitKey(25) & 0xFF == ord('q'):
        break

# Release the window and close all windows
cv2.destroyAllWindows()

Using cache found in C:\Users\admin/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2023-5-2 Python-3.10.11 torch-2.0.0 CUDA:0 (NVIDIA GeForce RTX 3090, 24575MiB)

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients


requirements: C:\Users\admin\.cache\torch\hub\requirements.txt not found, check failed.


Adding AutoShape... 


Model: "model_17"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_18 (InputLayer)          [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 block1_conv1 (Conv2D)          (None, 224, 224, 64  1792        ['input_18[0][0]']               
                                )                                                                 
                                                                                                  
 block1_conv2 (Conv2D)          (None, 224, 224, 64  36928       ['block1_conv1[0][0]']           
                                )                                                          

In [115]:
detected_video = np.array(detected_video)
size = detected_video.shape[2] , detected_video.shape[1]

fps = 30


out = cv2.VideoWriter('sh/mov/___new__'+ name+"_"+ str(r) +'reco.mp4', cv2.VideoWriter_fourcc(*'mp4v'), fps, size)

for f in detected_video:
    out.write(f)
out.release() 